# Try Text Sorting

In [9]:
import json
import numpy as np
from collections import defaultdict

def sort_ancient_text(ocr_result):
    """
    고문서 OCR 결과를 우→좌, 상→하 순서로 정렬
    """
    # OCR 결과에서 필요한 정보 추출
    boxes = ocr_result['rec_boxes']
    texts = ocr_result['rec_texts']
    
    # 박스와 텍스트를 묶어서 처리
    text_boxes = []
    for i, (box, text) in enumerate(zip(boxes, texts)):
        x1, y1, x2, y2 = box
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
        text_boxes.append({
            'text': text,
            'center_x': center_x,
            'center_y': center_y,
            'box': box,
            'index': i
        })
    
    # 1단계: 열(column) 그룹화
    # x 좌표 기준으로 클러스터링하여 열을 식별
    columns = group_into_columns(text_boxes)
    
    # 2단계: 각 열을 오른쪽에서 왼쪽 순서로 정렬
    columns.sort(key=lambda col: -np.mean([item['center_x'] for item in col]))
    
    # 3단계: 각 열 내에서 위에서 아래 순서로 정렬
    sorted_text = []
    for column in columns:
        column.sort(key=lambda item: item['center_y'])
        sorted_text.extend([item['text'] for item in column])
    
    return sorted_text

def group_into_columns(text_boxes, threshold=80):
    """
    텍스트 박스들을 x 좌표 기준으로 열(column)별로 그룹화
    개선된 클러스터링 알고리즘 사용
    """
    if not text_boxes:
        return []
    
    # 1단계: 초기 클러스터링
    clusters = []
    for box in text_boxes:
        # 기존 클러스터 중 가장 가까운 것 찾기
        best_cluster = None
        min_distance = float('inf')
        
        for cluster in clusters:
            cluster_center = np.mean([item['center_x'] for item in cluster])
            distance = abs(box['center_x'] - cluster_center)
            if distance < min_distance:
                min_distance = distance
                best_cluster = cluster
        
        # 임계값 내의 클러스터가 있으면 추가, 없으면 새 클러스터 생성
        if best_cluster is not None and min_distance <= threshold:
            best_cluster.append(box)
        else:
            clusters.append([box])
    
    # 2단계: 작은 클러스터들을 인근 큰 클러스터에 병합
    merged_clusters = []
    for cluster in sorted(clusters, key=len, reverse=True):
        if len(cluster) >= 2:  # 큰 클러스터는 그대로 유지
            merged_clusters.append(cluster)
        else:  # 작은 클러스터는 가장 가까운 큰 클러스터에 병합
            if merged_clusters:
                cluster_x = cluster[0]['center_x']
                best_target = min(merged_clusters, 
                                key=lambda c: abs(np.mean([item['center_x'] for item in c]) - cluster_x))
                best_target.extend(cluster)
            else:
                merged_clusters.append(cluster)
    
    return merged_clusters

def format_text_output(sorted_texts):
    """
    정렬된 텍스트를 읽기 좋은 형태로 포맷팅
    """
    result = ""
    for i, text in enumerate(sorted_texts, 1):
        result += f"{i:2d}. {text}\n"
    return result

# 사용 예시
def main():
    # JSON 파일 로드
    with open('/Users/user/paddleocrtest/output/태조실록_001권_총서_001a면_cropped_res.json', 'r', encoding='utf-8') as f:
        ocr_data = json.load(f)
    
    # 텍스트 정렬
    sorted_texts = sort_ancient_text(ocr_data)
    
    # 결과 출력
    print("=== 정렬된 고문서 텍스트 ===")
    print(format_text_output(sorted_texts))
    
    # 연결된 전체 텍스트
    print("\n=== 연결된 전체 텍스트 ===")
    full_text = "".join(sorted_texts)
    print(full_text)

if __name__ == "__main__":
    main()

=== 정렬된 고문서 텍스트 ===
 1. 生諱安社是為
 2. 太祖康献大王實録卷第一
 3. 景英生諱忠敏忠敏生諱华华生諱珍有珍有生諱宫進宫進
 4. 諱克谦之女生将军諱阳茂将軍娶上将军李公諱康濟之女
 5. 兢休生諱廉顺廉顺生諱承朔承朔生諱充慶充慶生諱景英
 6. 孫軍尹金殷義之女生侍中諱自延侍中生僕射諱天祥僕射
 7. 成桂號松軒全州大姓也有司空諱翰仕新罗娶太宗王十世
 8. 太祖康献至仁啓运聖文神武大王姓李氏諱旦字君晋古諱
 9. 生大将军諱勇夫大将军生内侍执奏諱降执奏娶侍中文公
10. 餘勇略過人山城别监入馆因官妓事與州官有隙州官與按
11. 生阿干諱光禧阿干生司徒三重大匡諱立全司徒生諱兢休
12. 穆祖性豪放有志四方初在全州时年二十
13. 廉議上聞發兵圖之
14. 從而徙者百七十餘家嘗造船十五隻以備倭既元也窟大王
15. 穆祖聞之遂徙居江陵道三陟縣民愿
16. 兵侵諸郡
17. 穆祖保頭陀山城以避配通前日山城别监新除
18. 按廉使又将至
19. 穆祖恐祸及挈家浮海至东北面宜州


=== 연결된 전체 텍스트 ===
生諱安社是為太祖康献大王實録卷第一景英生諱忠敏忠敏生諱华华生諱珍有珍有生諱宫進宫進諱克谦之女生将军諱阳茂将軍娶上将军李公諱康濟之女兢休生諱廉顺廉顺生諱承朔承朔生諱充慶充慶生諱景英孫軍尹金殷義之女生侍中諱自延侍中生僕射諱天祥僕射成桂號松軒全州大姓也有司空諱翰仕新罗娶太宗王十世太祖康献至仁啓运聖文神武大王姓李氏諱旦字君晋古諱生大将军諱勇夫大将军生内侍执奏諱降执奏娶侍中文公餘勇略過人山城别监入馆因官妓事與州官有隙州官與按生阿干諱光禧阿干生司徒三重大匡諱立全司徒生諱兢休穆祖性豪放有志四方初在全州时年二十廉議上聞發兵圖之從而徙者百七十餘家嘗造船十五隻以備倭既元也窟大王穆祖聞之遂徙居江陵道三陟縣民愿兵侵諸郡穆祖保頭陀山城以避配通前日山城别监新除按廉使又将至穆祖恐祸及挈家浮海至东北面宜州


# Text Sorting - Final

In [ ]:
import json
import numpy as np
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

def improved_sort_ancient_text(ocr_result, debug=False):
    """
    개선된 고문서 OCR 결과 정렬 (우→좌, 상→하)
    DBSCAN 클러스터링을 사용하여 더 정확한 열 구분
    """
    boxes = ocr_result['rec_boxes']
    texts = ocr_result['rec_texts']
    
    # 박스와 텍스트를 묶어서 처리
    text_boxes = []
    for i, (box, text) in enumerate(zip(boxes, texts)):
        x1, y1, x2, y2 = box
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
        height = y2 - y1
        text_boxes.append({
            'text': text,
            'center_x': center_x,
            'center_y': center_y,
            'height': height,
            'box': box,
            'index': i
        })
    
    if debug:
        print("=== 원본 데이터 분석 ===")
        for i, item in enumerate(text_boxes):
            print(f"{i:2d}: X={item['center_x']:4.0f} Y={item['center_y']:4.0f} H={item['height']:4d} \"{item['text'][:20]}...\"")
    
    # 1단계: DBSCAN을 사용한 열 클러스터링
    columns = cluster_columns_dbscan(text_boxes, debug)
    
    # 2단계: 각 열을 오른쪽에서 왼쪽 순서로 정렬
    columns.sort(key=lambda col: -np.mean([item['center_x'] for item in col]))
    
    # 3단계: 각 열 내에서 Y 좌표 기준 정렬 (상→하)
    sorted_text = []
    for col_idx, column in enumerate(columns):
        # 열 내에서 Y 좌표 순으로 정렬
        column.sort(key=lambda item: item['center_y'])
        
        if debug:
            avg_x = np.mean([item['center_x'] for item in column])
            print(f"\n열 {col_idx + 1} (평균 X: {avg_x:.0f}):")
            for j, item in enumerate(column):
                print(f"  {j+1:2d}: Y={item['center_y']:4.0f} \"{item['text'][:30]}...\"")
        
        sorted_text.extend([item['text'] for item in column])
    
    return sorted_text

def cluster_columns_dbscan(text_boxes, debug=False):
    """
    DBSCAN 클러스터링을 사용한 열 구분
    """
    # X 좌표만 사용하여 클러스터링
    X = np.array([[item['center_x']] for item in text_boxes])
    
    # DBSCAN 매개변수 조정
    # eps: 클러스터링 거리 임계값
    # min_samples: 최소 샘플 수
    eps = 120  # 픽셀 단위
    min_samples = 1
    
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
    labels = clustering.labels_
    
    # 클러스터별로 그룹화
    columns = {}
    for i, label in enumerate(labels):
        if label == -1:  # 노이즈로 분류된 경우 가장 가까운 클러스터에 할당
            label = find_nearest_cluster(text_boxes[i], columns)
        
        if label not in columns:
            columns[label] = []
        columns[label].append(text_boxes[i])
    
    if debug:
        print(f"\n=== DBSCAN 클러스터링 결과 (eps={eps}) ===")
        for label, items in columns.items():
            avg_x = np.mean([item['center_x'] for item in items])
            print(f"클러스터 {label}: {len(items)}개 항목, 평균 X={avg_x:.0f}")
    
    return list(columns.values())

def find_nearest_cluster(item, clusters):
    """
    노이즈로 분류된 항목을 가장 가까운 클러스터에 할당
    """
    if not clusters:
        return 0
    
    min_distance = float('inf')
    nearest_label = 0
    
    for label, cluster_items in clusters.items():
        cluster_center_x = np.mean([ci['center_x'] for ci in cluster_items])
        distance = abs(item['center_x'] - cluster_center_x)
        if distance < min_distance:
            min_distance = distance
            nearest_label = label
    
    return nearest_label

def analyze_text_layout(ocr_result):
    """
    텍스트 레이아웃 분석 및 시각화
    """
    boxes = ocr_result['rec_boxes']
    texts = ocr_result['rec_texts']
    
    # 중심 좌표 계산
    centers = [(box[0] + box[2]) / 2 for box in boxes]
    y_centers = [(box[1] + box[3]) / 2 for box in boxes]
    
    # 시각화
    plt.figure(figsize=(12, 8))
    plt.scatter(centers, y_centers, alpha=0.7)
    
    # 각 점에 인덱스 표시
    for i, (x, y) in enumerate(zip(centers, y_centers)):
        plt.annotate(f'{i}', (x, y), xytext=(5, 5), textcoords='offset points')
    
    plt.xlabel('X 좌표 (Center)')
    plt.ylabel('Y 좌표 (Center)')
    plt.title('고문서 텍스트 블록 분포')
    plt.gca().invert_yaxis()  # Y축 뒤집기 (이미지 좌표계)
    plt.grid(True, alpha=0.3)
    plt.show()
    
    return centers, y_centers

# 사용 예시
def main():
    # JSON 파일 로드
    with open('/Users/user/paddleocrtest/output/태조실록_001권_총서_001a면_cropped_res.json', 'r', encoding='utf-8') as f:
        ocr_data = json.load(f)
    
    # 레이아웃 분석 (선택사항)
    # analyze_text_layout(ocr_data)
    
    # 개선된 텍스트 정렬 (디버그 모드)
    sorted_texts = improved_sort_ancient_text(ocr_data, debug=False)

    def convert_ocr_results(ocr_result):
        import opencc
        converter = opencc.OpenCC('jp2t')
        converted_texts = [converter.convert(text) for text in ocr_result]
        return converted_texts
    
    # 결과 출력
    print("\n=== 최종 정렬된 텍스트 ===")
    for i, text in enumerate(sorted_texts, 1):
        print(f"{i:2d}. {text}")
    
    for i in range(len(sorted_texts)):
        con1 = "".join(convert_ocr_results(sorted_texts[i]))
        # con2 = "".join(convert_ocr_results(conv2, con1))
        print(f"{i:2d} : {con1}")

    # for i in range(len(sorted_texts)):
    #     print(f"changed {i} : {"".join(convert_ocr_results(sorted_texts[i]))}")
    # # 연결된 전체 텍스트
    print("\n=== 연결된 전체 텍스트 ===")
    full_text = "".join(sorted_texts)
    print(full_text)

if __name__ == "__main__":
    main()


=== 최종 정렬된 텍스트 ===
 1. 太祖康献大王實録卷第一
 2. 太祖康献至仁啓运聖文神武大王姓李氏諱旦字君晋古諱
 3. 成桂號松軒全州大姓也有司空諱翰仕新罗娶太宗王十世
 4. 孫軍尹金殷義之女生侍中諱自延侍中生僕射諱天祥僕射
 5. 生阿干諱光禧阿干生司徒三重大匡諱立全司徒生諱兢休
 6. 兢休生諱廉顺廉顺生諱承朔承朔生諱充慶充慶生諱景英
 7. 景英生諱忠敏忠敏生諱华华生諱珍有珍有生諱宫進宫進
 8. 生大将军諱勇夫大将军生内侍执奏諱降执奏娶侍中文公
 9. 諱克谦之女生将军諱阳茂将軍娶上将军李公諱康濟之女
10. 生諱安社是為
11. 穆祖性豪放有志四方初在全州时年二十
12. 餘勇略過人山城别监入馆因官妓事與州官有隙州官與按
13. 廉議上聞發兵圖之
14. 穆祖聞之遂徙居江陵道三陟縣民愿
15. 從而徙者百七十餘家嘗造船十五隻以備倭既元也窟大王
16. 兵侵諸郡
17. 穆祖保頭陀山城以避配通前日山城别监新除
18. 按廉使又将至
19. 穆祖恐祸及挈家浮海至东北面宜州
 0 : 太祖康獻大王實錄卷第一
 1 : 太祖康獻至仁啓运聖文神武大王姓李氏諱旦字君晉古諱
 2 : 成桂號松軒全州大姓也有司空諱翰仕新罗娶太宗王十世
 3 : 孫軍尹金殷義之女生侍中諱自延侍中生僕射諱天祥僕射
 4 : 生阿干諱光禧阿干生司徒三重大匡諱立全司徒生諱兢休
 5 : 兢休生諱廉顺廉顺生諱承朔承朔生諱充慶充慶生諱景英
 6 : 景英生諱忠敏忠敏生諱华华生諱珍有珍有生諱宫進宫進
 7 : 生大將军諱勇夫大將军生內侍执奏諱降执奏娶侍中文公
 8 : 諱克谦之女生將军諱阳茂將軍娶上將军李公諱康濟之女
 9 : 生諱安社是爲
10 : 穆祖性豪放有志四方初在全州时年二十
11 : 餘勇略過人山城别监入馆因官妓事與州官有隙州官與按
12 : 廉議上聞發兵圖之
13 : 穆祖聞之遂徙居江陵道三陟縣民愿
14 : 從而徙者百七十餘家嘗造船十五隻以備倭既元也窟大王
15 : 兵侵諸郡
16 : 穆祖保頭陀山城以避配通前日山城别监新除
17 : 按廉使又將至
18 : 穆祖恐祸及挈家浮海至东北面宜州

=== 연결된 전체 텍스트 ===
太祖康献大王實録卷第一太祖康献至仁啓运聖文神武大王姓李氏諱旦字君晋古諱成桂號松軒全州大姓也有司空諱翰仕新罗娶太宗王十世孫軍尹金殷義之女生侍

# Change Various Form(simplified ... etc)

In [ ]:
!pip install opencc

  Using cached opencc-1.1.9-cp313-cp313-macosx_11_0_arm64.whl
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import opencc
jp2t = opencc.OpenCC("jp2t")
s2t = opencc.OpenCC('s2t')
def convert_ocr_result(jp, simple, text):
    jp_txt = jp.convert(text)
    result = simple.convert(jp_txt)
    return result

aa =     "太祖康献至仁啓运聖文神武大王姓李氏諱旦字君晋古諱"
ocrred = "太祖康獻至仁啓运聖文神武大王姓李氏諱旦字君晉古諱"
target = "太祖康獻至仁啓運聖文神武大王姓李氏諱旦字君晋古諱"
result = convert_ocr_result(jp2t, s2t, aa)
target = convert_ocr_result(jp2t, s2t, aa)
print(f"{aa}\n{result}\n{target}\n{result == target}")

for i in range(len(result)):
    if result[i] != target[i]:
        print(result[i], target[i])

太祖康献至仁啓运聖文神武大王姓李氏諱旦字君晋古諱
太祖康獻至仁啓運聖文神武大王姓李氏諱旦字君晉古諱
太祖康獻至仁啓運聖文神武大王姓李氏諱旦字君晉古諱
True


In [78]:
convert_ocr_result(jp2t, jp2t, "太祖康獻大王實錄卷第一太祖康獻至仁啓運聖文神武大王姓李氏諱旦字君晉古諱成桂號松軒全州大姓也有司空諱翰仕新羅娶太宗王十世孫軍尹金殷義之女生侍中諱自延侍中生僕射諱天祥僕射生阿幹諱光禧阿幹生司徒三重大匡諱立全司徒生諱兢休兢休生諱廉順廉順生諱承朔承朔生諱充慶充慶生諱景英景英生諱忠敏忠敏生諱華華生諱珍有珍有生諱宮進宮進生大將軍諱勇夫大將軍生內侍執奏諱降執奏娶侍中文公諱克謙之女生將軍諱陽茂將軍娶上將軍李公諱康濟之女生諱安社是爲穆祖性豪放有志四方初在全州時年二十餘勇略過人山城別監入館因官妓事與州官有隙州官與按廉議上聞發兵圖之穆祖聞之遂徙居江陵道三陟縣民願從而徙者百七十餘家嘗造船十五隻以備倭既元也窟大王兵侵諸郡穆祖保頭陀山城以避配通前日山城別監新除按廉使又將至穆祖恐禍及挈家浮海至東北面宜州")

'太祖康獻大王實錄卷第一太祖康獻至仁啓運聖文神武大王姓李氏諱旦字君晉古諱成桂號松軒全州大姓也有司空諱翰仕新羅娶太宗王十世孫軍尹金殷義之女生侍中諱自延侍中生僕射諱天祥僕射生阿幹諱光禧阿幹生司徒三重大匡諱立全司徒生諱兢休兢休生諱廉順廉順生諱承朔承朔生諱充慶充慶生諱景英景英生諱忠敏忠敏生諱華華生諱珍有珍有生諱宮進宮進生大將軍諱勇夫大將軍生內侍執奏諱降執奏娶侍中文公諱克謙之女生將軍諱陽茂將軍娶上將軍李公諱康濟之女生諱安社是爲穆祖性豪放有志四方初在全州時年二十餘勇略過人山城別監入館因官妓事與州官有隙州官與按廉議上聞發兵圖之穆祖聞之遂徙居江陵道三陟縣民願從而徙者百七十餘家嘗造船十五隻以備倭既元也窟大王兵侵諸郡穆祖保頭陀山城以避配通前日山城別監新除按廉使又將至穆祖恐禍及挈家浮海至東北面宜州'

# save Sorted Text from JSON to CSV

In [ ]:
import json
import os
from typing import Tuple
import opencc
import csv
import re

# 사용 예시
def main():
    csv_data = []
    jp2t = opencc.OpenCC("jp2t")
    s2t = opencc.OpenCC('s2t')
    def parse_filename_info(json_filename: str, full_text: str) -> Tuple[str, str, str, str]:
        """
        파일명과 텍스트에서 정보 추출
        
        Args:
            json_filename: 예) 태조실록_001권_총서_002a면_cropped_res.json
            full_text: OCR로 추출된 전체 텍스트
        
        Returns:
            (original_filename, king, volume, page)
        """
        
        # 1. 원본 파일명 추출 (확장자를 .jpg로 변경)
        original_filename = json_filename.replace("_cropped_res.json", ".jpg")
        
        # 2. 파일명에서 정보 파싱
        # 예: 태조실록_001권_총서_002a면_cropped_res.json
        filename_parts = json_filename.replace("_cropped_res.json", "").split("_")
        
        volume = ""
        page = ""
        king_name = ""
        
        # 파일명 패턴 분석
        if len(filename_parts) >= 4:
            # [태조실록, 001권, 총서, 002a면] 형태
            king_record = filename_parts[0]  # 태조실록
            volume_part = filename_parts[1]   # 001권
            section_part = filename_parts[2]  # 총서
            page_part = filename_parts[3]     # 002a면
            
            volume = f"{king_record}_{volume_part}"  # 태조실록_001권
            page = f"{section_part}_{page_part}"     # 총서_002a면
            
            # 왕 이름 추출 (실록에서 XX 부분)
            king_match = re.match(r"(.+?)실록", king_record)
            # print(king_record, king_match)
            if king_match:
                king_name = king_match.group(1)  # 태조
            king_name = king_record[:4]
        if king_name:
            king = king_name
        else:
            king = "[왕 정보 없음]"
        
        return original_filename, king, volume, page

    def convert_ocr_result(jp, simple, text):
        jp_txt = jp.convert(text)
        result = simple.convert(jp_txt)
        return result
    # JSON 파일 로드
    for filename in os.listdir("output"):
        if not filename.endswith("json"):
            continue
        print(filename)
        with open(os.path.join("output", filename), "r", encoding = "utf-8") as f:
            ocr_data = json.load(f)

        sorted_texts = improved_sort_ancient_text(ocr_data, debug=False)
        for i in range(len(sorted_texts)):
            sorted_texts[i] = convert_ocr_result(jp2t, s2t, sorted_texts[i])
        full_text = "".join(sorted_texts)
        print(full_text)
        original_filename, king, volume, page = parse_filename_info(filename, full_text)
        csv_row = {
            'full_text': full_text,
            'original_filename': original_filename,
            'king': king,
            'volume': volume,
            'page': page
        }
        csv_data.append(csv_row)
    def get_sort_key(row):
        match = row['page'].split("_")[1][:4]
        if match:
            number = int(match[:3])
            suffix = match[3:]
            return (number, suffix)
        return (0, '')
    print(get_sort_key(csv_data[0]))
    csv_data.sort(key = get_sort_key)
    print(csv_data)
    fieldnames = ['king', 'volume', 'page', 'original_filename', 'full_text']
    with open("result.csv", 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # 헤더 작성
        writer.writeheader()
        
        # 데이터 작성
        for row in csv_data:
            writer.writerow(row)

if __name__ == "__main__":
    main()

태조실록_001권_총서_002a면_cropped_res.json
站在幹東管內故云然其平有大土城南京之平亦有大土城其北七八里又有大石城皆穆祖管領軍民之所居也穆祖維居幹東而往來諸城不常厥居幹東東南三十餘裏有海島曰者考羅北連於陸穆祖築石城以放牛馬憲宗八年受散吉令肯管領李春文大純趙奧魯哥兒卓青尚哉光奕張哥等八介百戶之任上充兼抗扎百戶句當世祖皇帝中統二年辛酉六月尚書省給降本研行使銅印至元元年甲子五月欽受宣命仍充幹東千戶句當至元十一年甲戌十二月薨葵於孔州奧興即慶城南五里後遷葬於鹹興府之義興部靼洞即德陵穆祖配孝妃李氏非一李也千牛衛長史諱公爾之女生諱行裏是爲翼祖至元十二年乙亥三月襲職十八年辛巳世祖徵日本天下兵船會於合浦翼祖蒙上司文字將本人戶簽搬軍人與雙城捻管府三撒千戶蒙古大塔失等赴徵遂見高麗忠烈王至於再三益恭益虔每謝曰先臣奔於北實脫虎狼之口耳非敢背君父也願上釋其罪王
태조실록_001권_총서_002b면_cropped_res.json
翼祖與夫人走馬至赤島北岸水廣可六百曰卿本士族豈忘本乎今觀卿舉止足知心之研存關初穆祖時時往峴城諸女真千戶達魯花赤皆願納交遂與之從遊諸千戶待甚厚必宰牛馬享宴輒留數日諸千戶有至斡東者穆祖亦如是逮翼祖承襲遵而不改翼祖威德漸靠本非我類今觀其勢終必不利於我請兵深處之人而除之且分其財產乎乃謬告曰吾等將獵北地而來請停會二十日翼祖許之過期不來翼祖親往峴城唯老弱婦女在丁壯無一人問之一女對曰貪其獸多至今不返耳翼祖乃還道見一老嫗頭戴水桶手持一碗翼祖忽渴欲飲老嫗淨洗其碗取水以進因言曰公不知乎此廢之人忌公將圖之請兵而去非獵也後三日必來貴官威德可惜不敢不告翼祖惶而返使家人船載家產順流豆滿江而下期會赤島自與孫夫人渡加陽灘登高望之則斡東之野賊彌淌而來先鋒三百餘人幾及之
태조실록_001권_총서_003b면_cropped_res.json
安邊府之瑞谷縣奉龍驛北洞即智陵度祖諱椿小字善來蒙古諱孛顏帖木兒受宣命襲職配敬妃樸氏斡東百戶贈門下侍中諱光之女生二男長曰子興蒙古名塔思不花次即我桓祖樸氏卒後移居和州娶趙氏雙城槐管之女生二男三女長完者不花次那海初翼祖以鹹州土地平行沃饒幹東之民南來者多廢之州之歸州草古臺王巨山雲天松豆等都連浦阿赤郎耳等地故稱鹹州爲幹東逸彥彥及是度祖盡有安邊以北之地而移居鹹州爲近南來之民